In [92]:
## a script to make a UMI count matrix from fastq
import gzip
import re
import os
import argparse
from multiprocessing import Pool

def processCellbarcode(barCodefile):
    '''takes in processed cell ranger barcode raw file 
    from processed folder /outs/raw_feature_bc_matrix/barcodes.tsv.gz and spits out a set of barcodes
    '''
    cellIDs = set()
    lineBuff = 0
    lineN = 0
    for sampleId in gzip.open(barCodefile, 'rb'):
        lineN += 1
        if lineN == 100000:
            lineBuff += 100000
            lineN =0
            print('PROCESSED LINES {} FROM {}'.format(lineBuff, barCodefile))
        s=re.sub('-.*', '', sampleId.decode().strip())
        cellIDs.add(s)
    return cellIDs

def readTable(R1path, R2path, outReadTable, cellIDs):
    cell, umi, tag= [0, 16], [16, 28], [0, 8]
    cell_start, cell_end = cell
    umi_start, umi_end = umi
    tag_start, tag_end = tag
    R1File = gzip.open(R1path, 'rb')
    R2File = gzip.open(R2path, 'rb')
    readTrue = 0
    outFile = open(outReadTable, 'w')
    outFile.write("Cell,Umi,Sample\n")
    for lineN, (r1, r2) in enumerate(zip(R1File, R2File)):
        if lineN % 1000000 == 0:
            print('{} PROCESSED LINES {}'.format(R1path, lineN))
        if lineN % 4 == 1: # every four lines
            fastaLineR1 = r1.strip()
            fastaLineR2 = r2.strip()
            if fastaLineR1 and fastaLineR2:
                cellNuc=fastaLineR1[cell_start:cell_end].decode()
                if cellNuc in cellIDs:
                    umiNuc=fastaLineR1[umi_start:umi_end].decode()
                    sampleNuc=fastaLineR2[tag_start:tag_end].decode()
                    outFile.write("{},{},{}\n".format(cellNuc,umiNuc,sampleNuc))
                    readTrue += 1
                    # print('CELL is {} \n UMI is {} \n & TAG is {}'.format(cellNuc, umiNuc, sampleNuc))
        lineN += 1
    print('PROCESSED {} FASTA RECORDS AND WROTE {} RECORDS TO {} readTable '.format(lineN//4, readTrue, outReadTable))
    outFile.close()

In [98]:
cellIDs = processCellbarcode("/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/10xRUN_cDNA1_3/outs/raw_feature_bc_matrix/barcodes.tsv.gz")

PROCESSED LINES 100000 FROM /labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/10xRUN_cDNA1_3/outs/raw_feature_bc_matrix/barcodes.tsv.gz
PROCESSED LINES 200000 FROM /labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/10xRUN_cDNA1_3/outs/raw_feature_bc_matrix/barcodes.tsv.gz
PROCESSED LINES 300000 FROM /labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/10xRUN_cDNA1_3/outs/raw_feature_bc_matrix/barcodes.tsv.gz
PROCESSED LINES 400000 FROM /labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/10xRUN_cDNA1_3/outs/raw_feature_bc_matrix/barcodes.tsv.gz
PROCESSED LINES 500000 FROM /labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/10xRUN_cDNA1_3/outs/raw_feature_bc_matrix/barcodes.tsv.gz
PROCESSED LINES 600000 FROM /labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/10xRUN_cDNA1_3/outs/raw_feature_bc_matrix/barcodes.tsv.gz
PROCESSED LINES 700000 FROM /labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/10xRUN_cDNA1_3/outs/raw_feature_bc_matrix/barcodes.tsv.gz
PROCESSED LIN

PROCESSED LINES 5900000 FROM /labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/10xRUN_cDNA1_3/outs/raw_feature_bc_matrix/barcodes.tsv.gz
PROCESSED LINES 6000000 FROM /labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/10xRUN_cDNA1_3/outs/raw_feature_bc_matrix/barcodes.tsv.gz
PROCESSED LINES 6100000 FROM /labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/10xRUN_cDNA1_3/outs/raw_feature_bc_matrix/barcodes.tsv.gz
PROCESSED LINES 6200000 FROM /labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/10xRUN_cDNA1_3/outs/raw_feature_bc_matrix/barcodes.tsv.gz
PROCESSED LINES 6300000 FROM /labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/10xRUN_cDNA1_3/outs/raw_feature_bc_matrix/barcodes.tsv.gz
PROCESSED LINES 6400000 FROM /labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/10xRUN_cDNA1_3/outs/raw_feature_bc_matrix/barcodes.tsv.gz
PROCESSED LINES 6500000 FROM /labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/10xRUN_cDNA1_3/outs/raw_feature_bc_matrix/barcodes.tsv.gz
PROCES

In [101]:
len(cellIDs)

6794880

In [102]:
multiseq_path = "/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16"

In [103]:
r1_paths = []
r2_paths = []
for path in os.listdir(multiseq_path):
    if path.startswith("Multi-seq"):
        if path.endswith("R1_001.fastq.gz"):
            r1_paths.append(path)
        elif path.endswith("R2_001.fastq.gz"):
            r2_paths.append(path)
r1_paths.sort()
r2_paths.sort()

In [104]:
def preprocess(paths):
    R1path, R2path = paths
    R1path, R2path = os.path.join(multiseq_path, R1path), os.path.join(multiseq_path, R2path)
    outReadTable = os.path.join(multiseq_path, R1path[:-16]+'_ReadTable.csv')
    readTable(R1path, R2path, outReadTable, cellIDs)

In [97]:
with Pool(32) as p:
    print(p.map(preprocess, zip(r1_paths, r2_paths)))

/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq6_S94_L004_R1_001.fastq.gz PROCESSED LINES 0
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq7_S95_L003_R1_001.fastq.gz PROCESSED LINES 0
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq4_S92_L003_R1_001.fastq.gz PROCESSED LINES 0
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq8_S96_L003_R1_001.fastq.gz PROCESSED LINES 0
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq3_S91_L004_R1_001.fastq.gz PROCESSED LINES 0
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq4_S92_L004_R1_001.fastq.gz PROCESSED LINES 0
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq6_S94_L003_R1_001.fastq.gz PROCESSED LINES 0
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq7_S95_L004_R1_001.fastq.gz PROCESSED LINES 0
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/Y

/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq3_S91_L004_R1_001.fastq.gz PROCESSED LINES 4000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq5_S93_L003_R1_001.fastq.gz PROCESSED LINES 4000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq4_S92_L004_R1_001.fastq.gz PROCESSED LINES 4000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq4_S92_L003_R1_001.fastq.gz PROCESSED LINES 4000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq5_S93_L004_R1_001.fastq.gz PROCESSED LINES 4000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq2_S90_L003_R1_001.fastq.gz PROCESSED LINES 4000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq2_S90_L004_R1_001.fastq.gz PROCESSED LINES 4000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq6_S94_L004_R1_001.fastq.gz PROCESSED LINES 4000000
/labs/mi

/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq7_S95_L004_R1_001.fastq.gz PROCESSED LINES 8000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq2_S90_L004_R1_001.fastq.gz PROCESSED LINES 8000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq6_S94_L004_R1_001.fastq.gz PROCESSED LINES 8000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq2_S90_L003_R1_001.fastq.gz PROCESSED LINES 8000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq4_S92_L003_R1_001.fastq.gz PROCESSED LINES 8000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq5_S93_L003_R1_001.fastq.gz PROCESSED LINES 8000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq7_S95_L003_R1_001.fastq.gz PROCESSED LINES 8000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L003_R1_001.fastq.gz PROCESSED LINES 8000000
/labs/mi

/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq7_S95_L003_R1_001.fastq.gz PROCESSED LINES 12000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq5_S93_L003_R1_001.fastq.gz PROCESSED LINES 12000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq5_S93_L004_R1_001.fastq.gz PROCESSED LINES 12000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq4_S92_L003_R1_001.fastq.gz PROCESSED LINES 12000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq3_S91_L003_R1_001.fastq.gz PROCESSED LINES 11000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L003_R1_001.fastq.gz PROCESSED LINES 12000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L004_R1_001.fastq.gz PROCESSED LINES 12000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq8_S96_L003_R1_001.fastq.gz PROCESSED LINES 13000000


/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq7_S95_L003_R1_001.fastq.gz PROCESSED LINES 16000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq5_S93_L004_R1_001.fastq.gz PROCESSED LINES 16000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq8_S96_L003_R1_001.fastq.gz PROCESSED LINES 17000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L004_R1_001.fastq.gz PROCESSED LINES 16000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq8_S96_L004_R1_001.fastq.gz PROCESSED LINES 17000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq6_S94_L003_R1_001.fastq.gz PROCESSED LINES 17000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L003_R1_001.fastq.gz PROCESSED LINES 16000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq2_S90_L003_R1_001.fastq.gz PROCESSED LINES 17000000


...

/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L003_R1_001.fastq.gz PROCESSED LINES 542000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L004_R1_001.fastq.gz PROCESSED LINES 548000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L003_R1_001.fastq.gz PROCESSED LINES 543000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L004_R1_001.fastq.gz PROCESSED LINES 549000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L003_R1_001.fastq.gz PROCESSED LINES 544000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L004_R1_001.fastq.gz PROCESSED LINES 550000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L003_R1_001.fastq.gz PROCESSED LINES 545000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L004_R1_001.fastq.gz PROCESSED LINES 55

/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L004_R1_001.fastq.gz PROCESSED LINES 581000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L003_R1_001.fastq.gz PROCESSED LINES 575000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L004_R1_001.fastq.gz PROCESSED LINES 582000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L003_R1_001.fastq.gz PROCESSED LINES 576000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L004_R1_001.fastq.gz PROCESSED LINES 583000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L003_R1_001.fastq.gz PROCESSED LINES 577000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L004_R1_001.fastq.gz PROCESSED LINES 584000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L003_R1_001.fastq.gz PROCESSED LINES 57

/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L003_R1_001.fastq.gz PROCESSED LINES 607000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L004_R1_001.fastq.gz PROCESSED LINES 615000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L003_R1_001.fastq.gz PROCESSED LINES 608000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L004_R1_001.fastq.gz PROCESSED LINES 616000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L003_R1_001.fastq.gz PROCESSED LINES 609000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L004_R1_001.fastq.gz PROCESSED LINES 617000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L003_R1_001.fastq.gz PROCESSED LINES 610000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L004_R1_001.fastq.gz PROCESSED LINES 61

/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L004_R1_001.fastq.gz PROCESSED LINES 648000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L003_R1_001.fastq.gz PROCESSED LINES 640000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L004_R1_001.fastq.gz PROCESSED LINES 649000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L003_R1_001.fastq.gz PROCESSED LINES 641000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L004_R1_001.fastq.gz PROCESSED LINES 650000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L003_R1_001.fastq.gz PROCESSED LINES 642000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L004_R1_001.fastq.gz PROCESSED LINES 651000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L003_R1_001.fastq.gz PROCESSED LINES 64

/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L004_R1_001.fastq.gz PROCESSED LINES 681000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L003_R1_001.fastq.gz PROCESSED LINES 673000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L004_R1_001.fastq.gz PROCESSED LINES 682000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L003_R1_001.fastq.gz PROCESSED LINES 674000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L003_R1_001.fastq.gz PROCESSED LINES 675000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L004_R1_001.fastq.gz PROCESSED LINES 683000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L003_R1_001.fastq.gz PROCESSED LINES 676000000
/labs/mignot/10xNMDA/191231_A00351_0308_BH2LW7DSXY_10x/YS-JY-16/Multi-seq1_S89_L004_R1_001.fastq.gz PROCESSED LINES 68